## Setup data for usage

Dataset contains one row per user for each day they were active. It complicates some queries, like Retention and Payer Retention, but it's an efficient way to record their activity.

In [1]:
import duckdb
from queries_and_graphs import *


# Connect to an in-memory database
con = duckdb.connect(':memory:')

# Read the CSV file
con.execute("CREATE TABLE dataset AS SELECT * FROM read_csv('data/dataset.csv')")


## Simulating Selections Bias

This can be done in a single SQL query. The query is available in `queries_and_graphs.py`.
We are mimicking Daily Purchase Rate and then flagging active users randomly. Flagging is equivalent to "fake purchase" event, so if non-flagged user is selected, they become flagged ("fake payer"), but if already flagged user is selected again, it's akin to repeated purchase and will not change user's state.
The reason why I consider Daily Purchase Rate more fundamental than, say, Conversion, is that it stabilizes over time which points to it being systemic response of the app-user system at scale.

In [2]:
df = con.execute(selection_bias_query).df()
df = df[(df['cohort_day'] > 0) & (df['cohort_day'] < 180)]

# Show Retention(s)
fig = show_retention(df)
fig.show()

# Show Conversion(s)
fig = show_cohort_conversion(df)
fig.show()

# Show Daily Purchase(s)
fig = show_daily_purchase(df)
fig.show()


It is evident that Selection Bias creates strong effect starting from cohort day 1, but we are selecting from a pool that's too large, which results in Cohort Conversion (flagged) that's more than twice the Cohort Conversion.

## Engagment Score and the Model

We'll introduce the model here in ad-hoc manner, without delving too deep in reasons behind specific choices.
We'll introduce two propensity scores Engagement and Payment Propensity as a single-parameter predictors of the corresponding behavior.

### Engagement score

If we suppose there's an inherit Egagement score that describes a user's propensity to engage with an app, it should be a good predictor of their long-term activity. Conversely, we can simply use long-term activity to calculate engagement score. Simplest model (that works) is calculating total Screen Time (time spent in the app) for the users in a dataset within some period after the registration and then simply setting Engagement to percentile rank.

We are using 60 days to calculate Engagement. It adds a bit accuracy, but using even 7 or less days yealds similar results, which is expected as activity is one of the best predictors of future activity. It goes to tell there's no much change over time in Engagement score.

### Payment Propensity

We'll introduce Payment Propensity analogly to Engagement. Now, the problem with Payment Propensity is that it can't be easily calculated from given data and the extent of problems will be shown later. But, what we can do in this pseudo-simulation is to arbitrarily assign Payment Propensity to each user. With `corr` correlation factor we can influence if we want to set it to be identical to Engagement, or completely random sampled from uniform distribution, or anything in between. As we are not interested in individual users, but group behavior, it can be shown that this approach works on a large enough sample.

### Cutoff - Payment Pool

From the above results it's clear that we aren't "hitting" already flagged users often enough. Cohort Conversion for flagged users grows to fast and too high. This parameter is introduced to limit this pool from which we are selecting.

It can be seen in real data that some users will be extremely active, but neve spend any money. An app can have users who are active for more than a year and never spend money. Interestingly, they are susceptible to discounts and given proper discount they can be converted, but will never, or very rarely, buy on original prices. This `cutoff` is therefore a parameter that's influenced by percieved value and drops with increasing value and/or reducing cost for user.

Interestingly, it seems that once the pool is defined, there's near equal probability for users from the pool to spend.

## First Test

Set `corr` to 0, which means Payment Propensity is random and unrelated to Engagement. We'll set `cuttoff` to a value that makes simulated Cohort Conversion Flagged similar to Cohort Conversion.

In [3]:
corr = 0
cutoff = 0.9
con.execute(engagement_score.format(corr=corr, days = 60))

df = con.execute(model_query.format(cutoff=cutoff)).df()
df = df[(df['cohort_day'] > 0) & (df['cohort_day'] < 180)]

# Show Retention(s)
fig = show_retention(df)
fig.show()

fig = show_cohort_conversion(df)
fig.show()

# fig = show_daily_purchase(df)
# fig.show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

There are a few things to note here. One is tha shapes or Cohort Conversion curves don't exactly match, but they are "close enough".

Second important thing is that now Flagged Retention is worse than Payer Retention (but still better than Retention). This indicates we do need `corr` greater than 0, because randomly generating Payment Propensity means there's equal probability that someone who is not engaged and someone who is very much engaged will spend money. This contradicts common sense, but it's clear from the results in this example as well.

## Second Test

We'll set `corr` to 1, which means we are selecting payers only from the most engaged group. In order for this to match Conversion Curve we need to reduce the pool significantly. This Cohort Conversion Flagged matches both scale and shape of Cohort Conversion, but it produces way higher Flagged Retention, much higher than Payer Retention. In fact, it's as high as >90% for the first month, with completely different shape.

This shows that no - your most engaged users aren't necessarily those who will convert.

In [4]:
corr = 1
cutoff = 0.9545
con.execute(engagement_score.format(corr=corr, days = 60))

df = con.execute(model_query.format(cutoff=cutoff)).df()
df = df[(df['cohort_day'] > 0) & (df['cohort_day'] < 180)]

# Show Retention(s)
fig = show_retention(df)
fig.show()

fig = show_cohort_conversion(df)
fig.show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Model Fitting

We now have a well parametrized model and data, we can fit the parameters. I've given an example of brute-force grid search here in order to show there is a solution and that performance function has one global minima.

In [9]:
import fit

fitness, x, y = fit.grid_search(correlation = [0, 1], cutoff_points = [0, 1], resolution = 0.1, silent=True)
fig = fit.heatmap(x, y, fitness)
fig.update_layout(height=600, width=600)
fig.show()

## Results

After couple of iterations we can settle on the following parameters for this particular dataset.

In [8]:
corr = 0.445
cutoff = 0.843
con.execute(engagement_score.format(corr=corr, days = 60))

df = con.execute(model_query.format(cutoff=cutoff)).df()
df = df[(df['cohort_day'] > 0) & (df['cohort_day'] < 180)]

# Show Retention(s)
fig = show_retention(df)
fig.show()

fig = show_cohort_conversion(df)
fig.show()

# fig = show_daily_purchase(df)
# fig.show()


These parameters fit both Payer Retention and Cohort Conversion very well, with the latter still suffering from the slightly different shape.

## Interpretation

There are several immidiate conclusions that this model points to.

### 1. Selection Bias

Selection Bias creates strong effect. In the end we've settled to `cutoff = 0.843`. This means that potential pool of payers is around 16%, yet only slightly more than 4% converted within 6 months. This implies that we are partly "paying" for high Payer Retention by not converting many potential payers who churn due to lower engagement before they hit the conversion point.

### 2. Engagement <-> Payment

There's a common wisdom that spending increases users engagement substantially. We've shown here that this is mostly an illusion created by selection bias, but not only that - we've gotten amazing fit without introducing any change in engagement from the value set at start.

Somewhat mismatched shape of Cohort Conversion does imply this effect exists as well and we'll add it in future iteration of this model, but it is also clear it is very small and shows visible change on a timescale of months, not immidiatelly.

### 3. Suprisingly low correlation

Correlation between Engagement and Payment Propensity is at the level of 0.4~0.5 for all the datasets I tested it, which was very surprising to me.

## Practical Implications

In the first place this model provides a correct way how to think about User Journey and how to correctly develop a strategy for different Engagement / Payment Propensity segments. 

It also enables to connect previously disparate metrics when making a decision and enables testing different monetization strategies not on a level of a single metric, but in a space of metrics that these parameters produce, with much deeper understanding of achieved effects.